# Выбор локации для скважины

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. 
Задача:
- Построить модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль.
- Проанализировать возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

Описание данных:

- 3 датасета с данными геологоразведки;
- id — уникальный идентификатор скважины;
- f0, f1, f2 — три признака точек;
- product — объём запасов в скважине (тыс. баррелей).

## Загрузка и подготовка данных

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

from sklearn.metrics import mean_squared_error
import math
import numpy as np
from scipy import stats as st

Загрузка датасетов.

In [2]:
df_1 = pd.read_csv('/datasets/geo_data_0.csv')
df_2 = pd.read_csv('/datasets/geo_data_1.csv')
df_3 = pd.read_csv('/datasets/geo_data_2.csv')

Открываю данные. Анализ.

In [3]:
df_1

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647
...,...,...,...,...,...
99995,DLsed,0.971957,0.370953,6.075346,110.744026
99996,QKivN,1.392429,-0.382606,1.273912,122.346843
99997,3rnvd,1.029585,0.018787,-1.348308,64.375443
99998,7kl59,0.998163,-0.528582,1.583869,74.040764


In [4]:
df_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [5]:
df_1.duplicated().sum()

0

In [6]:
df_1[df_1.duplicated('id',keep=False)]

,id,f0,f1,f2,product
931,HZww2,0.755284,0.368511,1.863211,30.681774
1364,bxg6G,0.411645,0.856830,-3.653440,73.604260
1949,QcMuo,0.506563,-0.323775,-2.215583,75.496502
3389,A5aEY,-0.039949,0.156872,0.209861,89.249364
7530,HZww2,1.061194,-0.373969,10.430210,158.828695
16633,fiKDv,0.157341,1.028359,5.585586,95.817889
21426,Tdehs,0.829407,0.298807,-0.049563,96.035308
41724,bxg6G,-0.823752,0.546319,3.630479,93.007798
42529,AGS9W,1.454747,-0.479651,0.683380,126.370504
51970,A5aEY,-0.180335,0.935548,-2.094773,33.020205


In [7]:
df_1 = df_1.drop('id', axis=1)

In [8]:
df_2

,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305
...,...,...,...,...,...
99995,QywKC,9.535637,-6.878139,1.998296,53.906522
99996,ptvty,-10.160631,-12.558096,5.005581,137.945408
99997,09gWa,-7.378891,-3.084104,4.998651,137.945408
99998,rqwUm,0.665714,-6.152593,1.000146,30.132364


In [9]:
df_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [10]:
df_2.duplicated().sum()

0

In [11]:
df_2[df_2.duplicated('id',keep=False)]

,id,f0,f1,f2,product
1305,LHZR0,11.170835,-1.945066,3.002872,80.859783
2721,bfPNe,-9.494442,-5.463692,4.006042,110.992147
5849,5ltQ6,-3.435401,-12.296043,1.999796,57.085625
41906,LHZR0,-8.989672,-4.286607,2.009139,57.085625
47591,wt4Uk,-9.091098,-8.109279,-0.002314,3.179103
82178,bfPNe,-6.202799,-4.820045,2.995107,84.038886
82873,wt4Uk,10.259972,-9.376355,4.994297,134.766305
84461,5ltQ6,18.213839,2.191999,3.993869,107.813044


In [12]:
df_2 = df_2.drop('id', axis=1)

In [13]:
df_3

,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746
...,...,...,...,...,...
99995,4GxBu,-1.777037,1.125220,6.263374,172.327046
99996,YKFjq,-1.261523,-0.894828,2.524545,138.748846
99997,tKPY3,-1.199934,-2.957637,5.219411,157.080080
99998,nmxp2,-2.419896,2.417221,-5.548444,51.795253


In [14]:
df_3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [15]:
df_3.duplicated().sum()

0

In [16]:
df_3[df_3.duplicated('id',keep=False)]

,id,f0,f1,f2,product
11449,VF7Jo,2.122656,-0.858275,5.746001,181.716817
28039,xCHr8,1.633027,0.368135,-2.378367,6.120525
43233,xCHr8,-0.847066,2.101796,5.597130,184.388641
44378,Vcm5J,-1.229484,-2.439204,1.222909,137.968290
45404,KUPhW,0.231846,-1.698941,4.990775,11.716299
49564,VF7Jo,-0.883115,0.560537,0.723601,136.233420
55967,KUPhW,1.211150,3.176408,5.543540,132.831802
95090,Vcm5J,2.587702,1.986875,2.482245,92.327572


In [17]:
df_3 = df_3.drop('id', axis=1)

Промежуточные выводы:
- Данные загружены.
- каждый датасет состоит из 100000 строк и 5 колонок.
- Есть скважины с одинаковым id. Более подробной информации о методе присвоения id нет. Тк количество повторяющихся id небольшое и данные в столбцах с признаками отличаются, строки не удалены.
- Убраны столбцы с id скважин, тк будут мешать обучению модели. Необходимости в них на данный момент нет.
- Типы данных верные.

## Обучение и проверка модели

Разобьем данные на обучающую и валидационные выборки для каждого региона.

In [18]:
RANDOM_STATE = 12345

1 регион

In [19]:
target_1 = df_1['product']
features_1 = df_1.drop('product', axis=1)

features_train_1, features_valid_1, target_train_1, target_valid_1 = train_test_split(
    features_1, target_1, test_size=0.25, random_state=RANDOM_STATE)

2 регион

In [20]:
target_2 = df_2['product']
features_2 = df_2.drop('product', axis=1)

features_train_2, features_valid_2, target_train_2, target_valid_2 = train_test_split(
    features_2, target_2, test_size=0.25, random_state=RANDOM_STATE)

3 региона

In [21]:
target_3 = df_3['product']
features_3 = df_3.drop('product', axis=1)

features_train_3, features_valid_3, target_train_3, target_valid_3 = train_test_split(
    features_3, target_3, test_size=0.25, random_state=RANDOM_STATE)

Обучим модель для каждого региона и предскажем объем запасов. Оценим адекватность модели по RMSE.

In [22]:
model = LinearRegression()
model.fit(features_train_1, target_train_1)
predictions_valid_1 = model.predict(features_valid_1)

rmse_1 = mean_squared_error(target_valid_1,predictions_valid_1)**0.5
mean_predicted_valid_1 = predictions_valid_1.mean()

print('Средний запас предсказанного сырья первого региона:', mean_predicted_valid_1)
print('Фактический средний запас сырья в первом регионе:', target_valid_1.mean())
print("RMSE модели линейной регрессии на валидационной выборке первого региона:", rmse_1)

Средний запас предсказанного сырья первого региона: 92.59256778438035
Фактический средний запас сырья в первом регионе: 92.07859674082927
RMSE модели линейной регрессии на валидационной выборке первого региона: 37.5794217150813


In [23]:
model = LinearRegression()
model.fit(features_train_2, target_train_2)
predictions_valid_2 = model.predict(features_valid_2)

rmse_2 = mean_squared_error(target_valid_2,predictions_valid_2)**0.5
mean_predicted_valid_2 = predictions_valid_2.mean()

print('Средний запас предсказанного сырья во втором регионе:', mean_predicted_valid_2)
print('Фактический средний запас сырья во втором регионе:', target_valid_2.mean())
print("RMSE модели линейной регрессии на валидационной выборке второго региона:", rmse_2)

Средний запас предсказанного сырья во втором регионе: 68.728546895446
Фактический средний запас сырья во втором регионе: 68.72313602435997
RMSE модели линейной регрессии на валидационной выборке второго региона: 0.893099286775617


In [24]:
model = LinearRegression()
model.fit(features_train_3, target_train_3)
predictions_valid_3 = model.predict(features_valid_3)

rmse_3 = mean_squared_error(target_valid_3,predictions_valid_3)**0.5
mean_predicted_valid_3 = predictions_valid_3.mean()

print('Средний запас предсказанного сырья в третьем регионе:', mean_predicted_valid_3)
print('Фактический средний запас сырья в третьем регионе:', target_valid_3.mean())
print("RMSE модели линейной регрессии на валидационной выборке третьего региона:", rmse_3)

Средний запас предсказанного сырья в третьем регионе: 94.96504596800489
Фактический средний запас сырья в третьем регионе: 94.88423280885438
RMSE модели линейной регрессии на валидационной выборке третьего региона: 40.02970873393434


Модель на втором регионе показала результат значительно лучше чем в других. Проверю дополнительно корреляцию между признаками и запасами.

In [25]:
df_1.corr()

,f0,f1,f2,product
f0,1.000000,-0.440723,-0.003153,0.143536
f1,-0.440723,1.000000,0.001724,-0.192356
f2,-0.003153,0.001724,1.000000,0.483663
product,0.143536,-0.192356,0.483663,1.000000


In [26]:
df_2.corr()

,f0,f1,f2,product
f0,1.000000,0.182287,-0.001777,-0.030491
f1,0.182287,1.000000,-0.002595,-0.010155
f2,-0.001777,-0.002595,1.000000,0.999397
product,-0.030491,-0.010155,0.999397,1.000000


In [27]:
df_3.corr()

,f0,f1,f2,product
f0,1.000000,0.000528,-0.000448,-0.001987
f1,0.000528,1.000000,0.000779,-0.001012
f2,-0.000448,0.000779,1.000000,0.445871
product,-0.001987,-0.001012,0.445871,1.000000


Во втором регионе в признаке f2 наблюдается корреляция практически равная 1 и в два раза больше чем в первом и третьем регионе. Этим обусловлен столь качественный результат.

Промежуточные выводы:
- Данные разделены на обучающие и валидационные выборки для каждого региона;
- Для каждого региона обучена модель и сделаны предсказания по объему запасов;
- Наимбольший предсказанный объем в 3 регионе;
- Несмотря на практически полное свопадение среднего объема запасов между предсказанными объемами и фактическими, наблюдается различие в RMSE.
- Высокое значение RMSE модели для 1 и 3 участка говорит о сильном различии между предсказанным результатом и фактическим.
- RMSE для 2 региона - 0.8, показывает, что модель хорошо справилась с предсказанием и результаты максимально близки к реальным.
- Результат предсказаний 2 модели обусловлен высоким уровнем корелляции между признаком f2 и целевым значением. 

## Подготовка к расчёту прибыли

Ввожу переменные бюджета и стоимости тысячи бареллей. Расчитываю необходимый объем для окупаемости.

In [28]:
BUDGET = 10000000000 #бюджет на разработку региона
COST_PRODUCT = 450000 #стоимость тысячи бареллей
PAYBACK = math.ceil(BUDGET / COST_PRODUCT) #окупаемость
WORKING_WELLS = 200 #кол-во работающих скважин

print ('Для безубыточной работы необходимо продать более', PAYBACK, 'тыс. бареллей')
print ('Окупаемость разработки месторождения, если на каждой скважине не менее', math.ceil(PAYBACK /WORKING_WELLS), 'тыс. бареллей.')

Для безубыточной работы необходимо продать более 22223 тыс. бареллей
Окупаемость разработки месторождения, если на каждой скважине не менее 112 тыс. бареллей.


Промежуточные вывоы:
 - Введены бизнес праметры необходимые для расчётов прибыли.
 - При стоимости 450 т.р. за тысячу бареллей необходимо продать 22 223 тыс бареллей.
 - При условии работы 200 скважин в одном регионе, на каждой скважине в среднем должно быть запасов не менее 112 тыс бареллей.
 - Результаты средних запасов во всех трех регионах предсказанные моделью выше меньше необходимых 112 т.б., 92, 68 и 64 т.б.

## Расчёт прибыли и рисков 

Сделаю формулу по расчёту прибыли с региона.

In [29]:
def income(target, probabilities, count):
    t = target.reset_index(drop = True)
    p = pd.Series(probabilities)
    probs_sorted = p.sort_values(ascending=False)
    selected = t[probs_sorted.index][:count]
    return selected.sum()  * COST_PRODUCT


Расчитаем доход для каждого региона по ранее созданной формуле.

In [30]:
profit_1 = income(target_valid_1, predictions_valid_1,200)
print(f'Выручка в первом регионе {profit_1/1000000000:.1f} млрд.р.')
print(f'Прибыль: {(profit_1 - BUDGET)/1000000000:.1f} млрд.р.')

Выручка в первом регионе 13.3 млрд.р.
Прибыль: 3.3 млрд.р.


In [31]:
profit_2 = income(target_valid_2, predictions_valid_2,200)
print(f'Выручка во вторм регионе {profit_2/1000000000:.1f} млрд. р.')
print(f'Прибыль: {(profit_2 - BUDGET)/1000000000:.1f} млрд.р.')

Выручка во вторм регионе 12.4 млрд. р.
Прибыль: 2.4 млрд.р.


In [32]:
profit_3 = income(target_valid_3, predictions_valid_3,200)
print(f'Выручка в третьем регионе {profit_3/1000000000:.1f} млрд. р.')
print(f'Прибыль: {(profit_3 - BUDGET)/1000000000:.1f} млрд.р.')

Выручка в третьем регионе 12.7 млрд. р.
Прибыль: 2.7 млрд.р.


Делаю бутстреп для выборки первого региона

In [33]:
STATE = np.random.RandomState(12345)

In [34]:
boot_t_1 = target_valid_1.reset_index(drop = True)
boot_p_1 = pd.Series(predictions_valid_1)

def revenue(target, probabilities, count):
    probs_sorted = probabilities.sort_values(ascending=False)
    selected = target[probs_sorted.index][:count]
    return (selected.sum()  * COST_PRODUCT)  - BUDGET

STATE = np.random.RandomState(12345)
    
values_1 = []
for i in range(1000):
    target_subsample = boot_t_1.sample(n=500, replace=True, random_state=STATE)
    probs_subsample = boot_p_1[target_subsample.index]
    rev = revenue(target_subsample,probs_subsample,200)
    values_1.append(rev)
    
values_1 = pd.Series(values_1)
lower_1 = values_1.quantile(0.025)
upper_1 = values_1.quantile(0.975)
mean_1 = values_1.mean()
print('Первый регион')
print(f'Средняя прибыль: {mean_1 / 1000000:.1f} млн. руб.')
print('Убыточных шахт:',(values_1 < 0).mean())
print(f'95%-ый доверительный интервал: от {lower_1/1000000:.1f} до {upper_1/1000000:.1f} млн. руб.')

Первый регион
Средняя прибыль: 425.9 млн. руб.
Убыточных шахт: 0.06
95%-ый доверительный интервал: от -102.1 до 948.0 млн. руб.


In [35]:
boot_t_2 = target_valid_2.reset_index(drop = True)
boot_p_2 = pd.Series(predictions_valid_2)

def revenue(target, probabilities, count):
    probs_sorted = probabilities.sort_values(ascending=False)
    selected = target[probs_sorted.index][:count]
    return (selected.sum()  * COST_PRODUCT) - BUDGET

STATE = np.random.RandomState(12345)
    
values_2 = []
for i in range(1000):
    target_subsample = boot_t_2.sample(n=500, replace=True, random_state=STATE)
    probs_subsample = boot_p_2[target_subsample.index]
    rev = revenue(target_subsample,probs_subsample,200)
    values_2.append(rev)
    
values_2 = pd.Series(values_2)
lower_2 = values_2.quantile(0.025)
upper_2 = values_2.quantile(0.975)
mean_2 = values_2.mean()
print('Второй регион')
print(f'Средняя прибыль: {mean_2 / 1000000:.1f} млн. руб.')
print('Убыточных шахт:',(values_2 < 0).mean())
print(f'95%-ый доверительный интервал: от {lower_2/1000000:.1f} до {upper_2/1000000:.1f} млн. руб.')

Второй регион
Средняя прибыль: 515.2 млн. руб.
Убыточных шахт: 0.01
95%-ый доверительный интервал: от 68.9 до 931.5 млн. руб.


In [36]:
boot_t_3 = target_valid_3.reset_index(drop = True)
boot_p_3 = pd.Series(predictions_valid_3)

def revenue(target, probabilities, count):
    probs_sorted = probabilities.sort_values(ascending=False)
    selected = target[probs_sorted.index][:count]
    return (selected.sum()  * COST_PRODUCT) - BUDGET

STATE = np.random.RandomState(12345)
    
values_3 = []
for i in range(1000):
    target_subsample = boot_t_3.sample(n=500, replace=True, random_state=STATE)
    probs_subsample = boot_p_3[target_subsample.index]
    rev = revenue(target_subsample,probs_subsample,200)
    values_3.append(rev)
    
values_3 = pd.Series(values_3)
lower_3 = values_3.quantile(0.025)
upper_3 = values_3.quantile(0.975)
mean_3 = values_3.mean()
print('третий регион')
print(f'Средняя прибыль: {mean_3 / 1000000:.1f} млн. руб.')
print('Убыточных шахт:',(values_3 < 0).mean())
print(f'95%-ый доверительный интервал: от {lower_3/1000000:.1f} до {upper_3/1000000:.1f} млн. руб.')

третий регион
Средняя прибыль: 435.0 млн. руб.
Убыточных шахт: 0.064
95%-ый доверительный интервал: от -128.9 до 969.7 млн. руб.


## Выводы

1. Открыты, проанализироаъваны и проверены данные с треёх регионов.
2. Проведена разбивка на две выборки в каждом регионе.
3. Обучена модель и сделаны предсказания для каждого региона.
 - Предсказан наибольший объем сырья - 3 регион, 94,8 млн. бар., RMSE = 40.
 - Самый точный прогноз - 2 регион,68,7 млн. бар., RMSE = 0,8
4. Обнаружена сильная корреляция между запасом нефти и признаком f2. Необходимо изучить отдельно качество и систему сбора данных по нему.
5. Создана формула для расчёта дохода с региона.
    - Самый прибыльный регион - Первый.
6. Проведён бутстреп для трёх регионов: максимальная среддняя прибыль - второй регион.

Рекомендация: Для разработки необходимо выбрать второй участок.

Обоснование: По условиям задачи, уровень риска должен быть ниже 2,5%. В доверительном инетрвале 1 и 3 региона присутсвуют отрицательные значения. Большая средняя прибыльность при минимальной убыточности у второго региона - 515 млн р.